<a href="https://colab.research.google.com/github/jinsusong/NLP-BERT-Review/blob/main/Huggingface_distilbert_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 데이터 증폭 

In [ ]:
!pip install numpy requests nlpaug
!pip install transformers

import pandas as pd
import numpy as np
import nlpaug.augmenter.word as nlpaw
from tqdm import tqdm



def augment_sentence(sentence, aug, num_threads):
    """""""""
    Constructs a new sentence via text augmentation.
    
    Input:
        - sentence:     A string of text
        - aug:          An augmentation object defined by the nlpaug library
        - num_threads:  Integer controlling the number of threads to use if
                        augmenting text via CPU
    Output:
        - A string of text that been augmented
    """""""""
    return aug.augment(sentence, num_thread=num_threads)
    


def augment_text(df, aug, num_threads, num_times):
    """""""""
    Takes a pandas DataFrame and augments its text data.
    
    Input:
        - df:            A pandas DataFrame containing the columns:
                                - 'comment_text' containing strings of text to augment.
                                - 'isToxic' binary target variable containing 0's and 1's.
        - aug:           Augmentation object defined by the nlpaug library.
        - num_threads:   Integer controlling number of threads to use if augmenting
                         text via CPU
        - num_times:     Integer representing the number of times to augment text.
    Output:
        - df:            Copy of the same pandas DataFrame with augmented data 
                         appended to it and with rows randomly shuffled.
    """""""""
    
    # Get rows of data to augment
    to_augment = df[df['isToxic']==1]
    to_augmentX = to_augment['comment_text']
    to_augmentY = np.ones(len(to_augmentX.index) * num_times, dtype=np.int8)
    
    # Build up dictionary containing augmented data
    aug_dict = {'comment_text':[], 'isToxic':to_augmentY}
    for i in tqdm(range(num_times)):
        augX = [augment_sentence(x, aug, num_threads) for x in to_augmentX]
        aug_dict['comment_text'].extend(augX)
    
    # Build DataFrame containing augmented data
    aug_df = pd.DataFrame.from_dict(aug_dict)
    
    return df.append(aug_df, ignore_index=True).sample(frac=1, random_state=42)
    

    
# Define nlpaug augmentation object 
aug10p = nlpaw.ContextualWordEmbsAug(model_path='bert-base-uncased', aug_min=1, aug_p=0.1, action="substitute")

# Upsample minority class ('isToxic' == 1) to create a roughly 50-50 class distribution
balanced_df = augment_text(downsampled_df, aug10p, num_threads=8, num_times=3)

### 텍스트 토큰화

In [ ]:
# 사전 훈련된 모델을 선택하고 나면 사람이 읽을 수 있는 텍스트 문자열을 모델이 해석할 수 있는 형식으로 변환해야 함
# 이 프로세스를 토큰화 라고 하며 직관적인 Hugging Face API를 사용하면 단어와 문장 → 토큰 시퀀스 → 텐서로 변환되어 모델에 제공될 수 있는 숫자 시퀀스를 매우 쉽게 변환할 수 있음
# 토크나이저가 토큰화 체계에 [CLS] 및 [SEP]와 같은 특수 토큰을 포함하는 방법을 참조
# 선택한 모델에 특정한 토크나이저 객체. 에서 사용하는 토크나이저를 가져오기 위해 distilbert-base-uncased모델의 이름을 DistilBertTokenizerFast. 클래스의 .from_pretrained()메서드에 전달합니다 

from transformers import DistilBertTokenizerFast

# Instantiate DistilBERT tokenizer...we use the Fast version to optimize runtime
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
# 토크아니저 객체를 인스턴스화하면 토크나이저의 .batch_encode_plus() 방법을 사용하여 훈련, 검증 및 테스트 세트를 일괄적으로 인코딩할 수 있습니다. 


# Define the maximum number of words to tokenize (DistilBERT can tokenize up to 512)
MAX_LENGTH = 128


# Define function to encode text data in batches
def batch_encode(tokenizer, texts, batch_size=256, max_length=MAX_LENGTH):
    """""""""
    A function that encodes a batch of texts and returns the texts' corresponding encodings and attention masks that are ready to be fed into a pre-trained transformer model.
    
    Input:
        - tokenizer:   Tokenizer object from the PreTrainedTokenizer Class
        - texts:       List of strings where each string represents a text
        - batch_size:  Integer controlling number of texts in a batch
        - max_length:  Integer controlling max number of words to tokenize in a given text
    Output:
        - input_ids:       sequence of texts encoded as a tf.Tensor object
        - attention_mask:  the texts' attention mask encoded as a tf.Tensor object
    """""""""
    
    input_ids = []
    attention_mask = []

    # 인수 설정
    # max_length → 주어진 텍스트에서 토큰화할 최대 단어 수를 제어합니다.
    # padding → '가장 긴'으로 설정하면 배치에서 가장 긴 순서로 채웁니다.
    # truncation→ True 이면 max_length 에서 설정한 값에 따라 텍스트를 자릅니다
    # return_attention_mask→ True 이면 attention mask를 반환, 선택 사항이지만 attention mask는 가중치를 높게 봐야하는 토큰과 무시해야 하는 토큰(패딩의 경우)을 모델에 알려준다. 모델에 대한 입력으로 주의 마스크를 포함하면 모델 성능이 향상될 수 있다.
    # return_token_type_ids→ True 이면 토큰 유형 ID 를 반환,  토큰 ID가 입력의 한 시퀀스가 ​​끝나고 다른 시퀀스가 ​​시작하는 위치를 모델에 알려주기 때문에 입력으로 여러 시퀀스가 ​​필요한 부분에 작업이 필요합니다(예: 질문 응답에는 '질문' 및 '답변' 시퀀스가 ​​필요함). 그러나 분류 작업에는 입력으로 하나의 시퀀스만 필요하기 때문에 이것은 선택 사항이다..
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer.batch_encode_plus(batch,
                                             max_length=max_length,
                                             padding='longest', #implements dynamic padding
                                             truncation=True,
                                             return_attention_mask=True,
                                             return_token_type_ids=False
                                             )
        input_ids.extend(inputs['input_ids'])
        attention_mask.extend(inputs['attention_mask'])
    
    
    return tf.convert_to_tensor(input_ids), tf.convert_to_tensor(attention_mask)
    
    
# Encode X_train
X_train_ids, X_train_attention = batch_encode(tokenizer, X_train.tolist())

# Encode X_valid
X_valid_ids, X_valid_attention = batch_encode(tokenizer, X_valid.tolist())

# Encode X_test
X_test_ids, X_test_attention = batch_encode(tokenizer, X_test.tolist())


#  batch_encode 함수는 다음을 반환한다.
# input_ids → 일련의 숫자로 인코딩된 텍스트의 단어.
# attention_mask→ input_ids주의를 기울여야 할 숫자 와 무시해야 할 숫자를 모델에 알려주는 이진 시퀀스 (패딩의 경우).
# input_ids 및 attention_mask  둘 다 Tensorflow tf.Tensor 객체로 변환되어 모델에 입력으로 쉽게 제공될 수 있다.

### 모델 아키텍처 정의

- 훈련, 검증 및 테스트 세트를 인코딩했으므로 이제 모델 아키텍처를 정의할 차례
- 기본 모델로 DistilBERT를 사용할 것이므로  Hugging Face 라이브러리에서 distilbert-base-uncased 가져오기


- Hugging Face API가 DistilBERT의 구성 클래스에서 여러 인수를 변경하여 기본 모델 아키텍처를 조정할 수 있는 config 옵션을 제공한다

- DistilBERT의 구성 클래스를 수정한 후, 모델 이름과 구성 개체를 TFDistilBertModel클래스 의 .from_pretrained()메서드에 전달하여 특정 헤드 없이 기본 DistilBERT 모델을 TFDistilBertForSequenceClassification 인스턴스화 할 수 있다.

- DistilBERT의 사전 훈련된 가중치는 우리 모델의 기초 역할을 하기 때문에 모델이 추가된 분류 계층에 대해 합리적인 가중치를 학습하기 시작할 때 훈련의 초기 단계에서 업데이트를 방지하고 보존하고자 한다.

- DistillBERT의 사전 훈련된 가중치를 일시적으로 고정하려면 각 DistilBERT의 레이어에 대해 layer.trainable = False 설정 하고 나중에 모델 성능이 수렴되면 layer.trainable = True 설정하여 고정을 해제할 수 있다.

In [ ]:
# 기본 모델 초기화 
from transformers import TFDistilBertModel, DistilBertConfig

DISTILBERT_DROPOUT = 0.2
DISTILBERT_ATT_DROPOUT = 0.2
 
# Configure DistilBERT's initialization
config = DistilBertConfig(dropout=DISTILBERT_DROPOUT, 
                          attention_dropout=DISTILBERT_ATT_DROPOUT, 
                          output_hidden_states=True)
                          
# The bare, pre-trained DistilBERT transformer model outputting raw hidden-states 
# and without any specific head on top.
distilBERT = TFDistilBertModel.from_pretrained('distilbert-base-uncased', config=config)

# Make DistilBERT layers untrainable
for layer in distilBERT.layers:
    layer.trainable = False

### Add a Classification Head

- 모델 아키텍처를 구축하면서 얻은 DistilBERT's embedding layer 위에 classification head 를 추가할 것,  line 35. 실제로 모델의 출력은 다음을 포함하는 튜플이다. 

In [ ]:
# last_hidden_state→ 단어 수준 임베딩 shape ( batch_size , sequence_length , hidden_size =768).
# hidden_states→ [선택 사항] tf.Tensor 튜플 형태 (임베딩 출력용 1개 + 각 레이어 출력용 1개 ) ( batch_size , sequence_length , hidden_size = 768) , 구성 파일에서  output_hidden_states=True  설정할 때 반환됩니다 .
# attentions→ [선택 사항] Attention softmax 이후의 Attention의 가중치는 self-attention head에서 가중 평균을 계산하는 데 사용됩니다. 구성 파일에서 output_attentions=True 설정할 때 반환됩니다 .


MAX_LENGTH = 128
LAYER_DROPOUT = 0.2
LEARNING_RATE = 5e-5
RANDOM_STATE = 42

def build_model(transformer, max_length=MAX_LENGTH):
    """""""""
    Template for building a model off of the BERT or DistilBERT architecture
    for a binary classification task.
    
    Input:
      - transformer:  a base Hugging Face transformer model object (BERT or DistilBERT)
                      with no added classification head attached.
      - max_length:   integer controlling the maximum number of encoded tokens 
                      in a given sequence.
    
    Output:
      - model:        a compiled tf.keras.Model with added classification layers 
                      on top of the base pre-trained model architecture.
    """"""""""
    
    # Define weight initializer with a random seed to ensure reproducibility
    weight_initializer = tf.keras.initializers.GlorotNormal(seed=RANDOM_STATE) 
    
    # Define input layers
    input_ids_layer = tf.keras.layers.Input(shape=(max_length,), 
                                            name='input_ids', 
                                            dtype='int32')
    input_attention_layer = tf.keras.layers.Input(shape=(max_length,), 
                                                  name='input_attention', 
                                                  dtype='int32')
    
    # DistilBERT outputs a tuple where the first element at index 0
    # represents the hidden-state at the output of the model's last layer.
    # It is a tf.Tensor of shape (batch_size, sequence_length, hidden_size=768).
    last_hidden_state = transformer([input_ids_layer, input_attention_layer])[0]
    # 다른 레이어의 숨겨진 상태에 액세스하는 대신 last_hidden_state 이 튜플의 인덱스 0을 선택하여 액세스합니다 . 이 임베딩을 구축하면 일반적으로 최상의 결과를 얻을 수 있기 때문입니다.
    
    # We only care about DistilBERT's output for the [CLS] token, 
    # which is located at index 0 of every encoded sequence.  
    # Splicing out the [CLS] tokens gives us 2D data.
    cls_token = last_hidden_state[:, 0, :]
    
    ##                                                 ##
    ## Define additional dropout and dense layers here ##
    ##                                                 ##
    
    # Define a single node that makes up the output layer (for binary classification)
    output = tf.keras.layers.Dense(1, 
                                   activation='sigmoid',
                                   kernel_initializer=weight_initializer,  
                                   kernel_constraint=None,
                                   bias_initializer='zeros'
                                   )(cls_token)
    # 모델 성능에 대한 기준선을 얻으려면 [CLS] 토큰의 문장 수준 임베딩 위에 sigmoid 활성화 기능이 있는 단일 조밀한 출력 레이어를 추가하는 것으로 시작할 수 있습니다 . 
    
    # Define the model
    model = tf.keras.Model([input_ids_layer, input_attention_layer], output)
    
    # Compile the model
    model.compile(tf.keras.optimizers.Adam(lr=LEARNING_RATE), 
                  loss=focal_loss(),
                  metrics=['accuracy'])
    
    
    # 마지막으로 adam optimizer 의 학습률을 5e-5로 설정 하여 모델을 컴파일합니다 
    # (원본 BERT 논문의 저자는 학습률 3e-4, 1e-4, 5e-5, 3e-5를 좋은 시작점으로 권장합니다. ) 데이터 세트의 클래스 불균형을 적절하게 처리하기 위해 손실 함수 를 이진 교차 엔트로피 대신 초점 손실로 설정했습니다 .
    return model

###Training Classification Layer Weights

- 모델을 구축했으므로 이제 모델 성능이 수렴할 때까지 classification layer의 무작위로 초기화된 가중치를 훈련할 수 있습니다.

- 단일 출력 레이어만 있는 단순한 기준 모델의 경우 6개의 epoch에 걸쳐 768개의 사용 가능한 가중치를 모두 훈련하면(DistilBERT의 가중치가 모두 고정되므로) 테스트에서 85.7% 의 정확도 와 0.926 의 AUC-ROC 점수가 나옵니다 .

In [ ]:
EPOCHS = 6
BATCH_SIZE = 64
NUM_STEPS = len(X_train.index) // BATCH_SIZE

# Train the model
train_history1 = model.fit(
    x = [X_train_ids, X_train_attention],
    y = y_train.to_numpy(),
    epochs = EPOCHS,
    batch_size = BATCH_SIZE,
    steps_per_epoch = NUM_STEPS,
    validation_data = ([X_valid_ids, X_valid_attention], y_valid.to_numpy()),
    verbose=2
)

 - 몇줄의 코드로 훈련된 모델의 경우 나쁘지 않지만, 우리는 확실히 더 잘할 수 있으며, 이 단계에서 생각할 수 있는 한 가지는 모델 아키텍처를 변경하는 것
 
 - 결국 우리 모델은 DistilBERT 위에 하나의 출력 레이어만 있는 매우 간단하므로 그 사이에 additional dense 또는 dropout layers 를 추가하는 것이 좋다.

Comet.ml API를 사용하여 그리드 검색을 수행하여 두 개의 조밀한 계층에 대해 이 작업을 수행하고 특정 분류 문제에 대한 최적의 모델 아키텍처가 다음과 같은지 확인합니다.

[DistilBERT CLS Embedding Layer] + [Dense 256] + [Dense 32] + [Single-node Output Layer]

이러한 변경의 결과로 우리의 새 모델 은 추가된 분류 계층만 훈련함으로써 테스트 세트에서 87.3% 의 정확도 와 0.930 의 AUC-ROC를 기록했습니다

### DistillBERT 미세 조정 및 모든 가중치 훈련 

추가된 classification layer 훈련을 마치면 DitilBERT의 임베딩 계층을 고정 해제하고 모든 가중치를 더 낮은 학습 속도로 미세 조정하여 모델에서 더 많은 성능을 끌어낼 수 있습니다

고정 가중치 계층 취소 후 우리의 모델을 재 컴파일 할 필요가 있지만 그것을 제외하고는 훈련 절차는 이전 단계와 동일하다.

In [ ]:
FT_EPOCHS = 4
BATCH_SIZE = 64
NUM_STEPS = len(X_train.index)

# Unfreeze distilBERT layers and make available for training
for layer in distilBERT.layers:
    layer.trainable = True
    
# Recompile model after unfreezing
model.compile(optimizer=tf.keras.optimizers.Adam(lr=2e-5), 
              loss=focal_loss(),
              metrics=['accuracy'])

# Train the model
train_history2 = model.fit(
    x = [X_train_ids, X_train_attention],
    y = y_train.to_numpy(),
    epochs = FT_EPOCHS,
    batch_size = BATCH_SIZE,
    steps_per_epoch = NUM_STEPS,
    validation_data = ([X_valid_ids, X_valid_attention], y_valid.to_numpy()),
    verbose=2
)

In [ ]:
DistillBERT의 사전 훈련된 가중치를 미세 조정한 결과, 우리 모델 은 테스트 세트에서 92.18% 의 최종 정확도 와 0.969 의 AUC-ROC를 달성

### 결론 

In [ ]:
DistilBERT가 충분히 빠르지 않은 것처럼 ONNX Runtime을 사용 하여 가중치 양자화 및 모델 서빙을 사용 하여 추론 속도를 더욱 최적화할 수 있음. 추가적인 공부 필요
